In [3]:
import csv

from pathlib import Path

from google.cloud import bigquery
from google.oauth2 import service_account

In [6]:
## create service account from BQ_CREDS file
credentials = service_account.Credentials.from_service_account_file("..\.secrets_\BQ_CREDS.json")
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [7]:
def get_monthly_stats(year, month):
    QUERY = f"""
        SELECT
          COUNT(*) AS num_downloads,
          REGEXP_EXTRACT(file.project, r".*?-(.*?)-(?:.*-)?stubs") AS port,
          REGEXP_EXTRACT(file.project, r".*?-.*?-(?:(.*)-)?stubs") AS board,
          REGEXP_EXTRACT(file.version, r"(.*).post") AS version,
          DATE({year},{month},1) as report_date,
          file.project as project,
          file.version AS version_full,
          -- REGEXP_EXTRACT(file.version, r".*.post(.*)") AS post,
        FROM
          `bigquery-public-data.pypi.file_downloads`
        WHERE
          file.PROJECT LIKE 'micropython-%-stubs' -- Only query the previous month OF history
          AND DATE(timestamp) BETWEEN DATE({year},{month},1)
          AND DATE_ADD(DATE({year},{month},1), INTERVAL 1 MONTH)
          AND details.installer.name <> 'bandersnatch'
        GROUP BY
          port,
          board,
          version,
          project,
          version_full
          -- ,post
        ORDER BY
          `num_downloads` DESC
        """
    query_job = client.query(QUERY)  # API request

    results = query_job.result()  # Waits for query to finish
    return results

In [8]:
stats_list = []

year = 2023

for month in range(6, 8):
    print(f"Processing {year}-{month} ... ", end="", flush=True)
    results = get_monthly_stats(year, month)
    field_names = [f.name for f in results.schema]
    # print(field_names)
    print(f"Retrieved {results.total_rows} download summaries")
    for row in results:
        stats_list.append(dict(row))
    print()

Processing 2023-6 ... 

Retrieved 76 download summaries

Processing 2023-7 ... Retrieved 35 download summaries



In [9]:
stats_list

[{'num_downloads': 284,
  'port': 'stdlib',
  'board': None,
  'version': None,
  'report_date': datetime.date(2023, 6, 1),
  'project': 'micropython-stdlib-stubs',
  'version_full': '0.9.0'},
 {'num_downloads': 111,
  'port': 'stm32',
  'board': None,
  'version': '1.20.0',
  'report_date': datetime.date(2023, 6, 1),
  'project': 'micropython-stm32-stubs',
  'version_full': '1.20.0.post1'},
 {'num_downloads': 100,
  'port': 'esp32',
  'board': None,
  'version': '1.20.0',
  'report_date': datetime.date(2023, 6, 1),
  'project': 'micropython-esp32-stubs',
  'version_full': '1.20.0.post1'},
 {'num_downloads': 52,
  'port': 'rp2',
  'board': 'pico-w',
  'version': '1.20.0',
  'report_date': datetime.date(2023, 6, 1),
  'project': 'micropython-rp2-pico-w-stubs',
  'version_full': '1.20.0.post1'},
 {'num_downloads': 48,
  'port': 'rp2',
  'board': 'pico',
  'version': '1.20.0',
  'report_date': datetime.date(2023, 6, 1),
  'project': 'micropython-rp2-pico-stubs',
  'version_full': '1.20.0.

In [11]:
# read the current data from the csv file
csv_file = Path(".") / "downloads.csv"
with open(csv_file, "r") as f:
    reader = csv.DictReader(f)
    current_stats_list = [row for row in reader]

current_stats_list

[{'num_downloads': '110',
  'port': 'rp2',
  'board': '',
  'version': '1.18',
  'report_date': '2022-06-01',
  'project': 'micropython-rp2-stubs',
  'version_full': '1.18.post1'},
 {'num_downloads': '90',
  'port': 'esp8266',
  'board': '',
  'version': '1.18',
  'report_date': '2022-06-01',
  'project': 'micropython-esp8266-stubs',
  'version_full': '1.18.post1'},
 {'num_downloads': '90',
  'port': 'esp32',
  'board': '',
  'version': '1.18',
  'report_date': '2022-06-01',
  'project': 'micropython-esp32-stubs',
  'version_full': '1.18.post1'},
 {'num_downloads': '89',
  'port': 'stm32',
  'board': '',
  'version': '1.18',
  'report_date': '2022-06-01',
  'project': 'micropython-stm32-stubs',
  'version_full': '1.18.post1'},
 {'num_downloads': '87',
  'port': 'esp32',
  'board': '',
  'version': '1.19.1',
  'report_date': '2022-06-01',
  'project': 'micropython-esp32-stubs',
  'version_full': '1.19.1.post3'},
 {'num_downloads': '80',
  'port': 'esp8266',
  'board': '',
  'version': '

In [12]:
full_list = current_stats_list + stats_list

In [13]:
print("writing to csv")
keys = full_list[0].keys()

with open(csv_file, "w", newline="") as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()

    dict_writer.writerows(full_list)

writing to csv
